<a href="https://colab.research.google.com/github/kai-v/ImageMorphing/blob/main/ImageMorphing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mp
from skimage import data
from skimage import exposure
from skimage.exposure import match_histograms
import PIL as pl
from PIL import Image
import cv2